In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("RSA Data - 2024.xlsx")

In [3]:
print(df.columns)

Index(['File No', 'File Creat', 'Callin Date Time', 'Man year', 'Event',
       'Breakdown Location', 'Breakdown street', 'Near To', 'City', 'State',
       'Reason Of Breakdown', 'Damage Details', 'Location', 'Actual KM',
       'Final Service', 'lat', 'lon', 'Status Color', 'Start Date\n',
       'End Date', 'Postal Code', 'Search', 'Return Remark', 'Garage Town',
       'External Claim Number', 'ASSISTANCE VEHICLE NO', 'Dealer LAT',
       'Dealer LON', 'Garage Dealer Code', 'Vehicle Type ', 'GearBox',
       'Engine Type '],
      dtype='object')


In [4]:
import pandas as pd

# Assuming `df` is your DataFrame
columns_to_remove_exact = [
    'File No', 'Callin Date Time', 'Man year', 'Breakdown Location', 
    'Damage Details', 'Location', 'Actual KM', 'Postal Code', 'Return Remark',
    'Start Date\n', 'End Date'
]

# Additional columns to remove based on similarity
columns_to_remove_similar = [
    'Breakdown street', 'Near To', 'Final Service', 'Search', 
    'External Claim Number', 'ASSISTANCE VEHICLE NO', 'Dealer LAT', 
    'Dealer LON', 'Garage Dealer Code', 'Vehicle Type'
]

# Combine all columns to remove
columns_to_remove = columns_to_remove_exact + columns_to_remove_similar

# Drop the columns if they exist in the DataFrame
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])

# Display the updated DataFrame
print(df.head())


  File Creat        Event       City        State Reason Of Breakdown  \
0 2024-01-01  Pilot error  Mangaluru    Karnataka           FLAT TYRE   
1 2024-01-01  Pilot error    Kolkata  West Bengal  Battery Discharged   
2 2024-01-01    Breakdown     Mysuru    Karnataka       FUEL LEAKEAGE   
3 2024-01-01     Accident   Amritsar       Punjab            ACCIDENT   
4 2024-01-01    Breakdown      Osian    Rajasthan         BROKEN AXLE   

        lat       lon Status Color Garage Town       Vehicle Type  GearBox  \
0  74.83941  12.88253        Green         NaN                 CWA  Manual   
1  88.37579  22.48166        Green         NaN  Lady driven - Solo  Manual   
2  76.62982  12.29635        Green      Mysore   Normal Assistance  Manual   
3  74.86166  31.65801        Green    AMRITSAR   Normal Assistance  Manual   
4  72.90472  26.72454        Green     JODHPUR   Normal Assistance  Manual   

  Engine Type   
0       Petrol  
1       Petrol  
2       Petrol  
3       Diesel  
4      

In [5]:
df.head()

,File Creat,Event,City,State,Reason Of Breakdown,lat,lon,Status Color,Garage Town,Vehicle Type,GearBox,Engine Type
0,2024-01-01,Pilot error,Mangaluru,Karnataka,FLAT TYRE,74.83941,12.88253,Green,NaN,CWA,Manual,Petrol
1,2024-01-01,Pilot error,Kolkata,West Bengal,Battery Discharged,88.37579,22.48166,Green,NaN,Lady driven - Solo,Manual,Petrol
2,2024-01-01,Breakdown,Mysuru,Karnataka,FUEL LEAKEAGE,76.62982,12.29635,Green,Mysore,Normal Assistance,Manual,Petrol
3,2024-01-01,Accident,Amritsar,Punjab,ACCIDENT,74.86166,31.65801,Green,AMRITSAR,Normal Assistance,Manual,Diesel
4,2024-01-01,Breakdown,Osian,Rajasthan,BROKEN AXLE,72.90472,26.72454,Green,JODHPUR,Normal Assistance,Manual,Diesel


In [6]:
df.to_csv("cleaned.csv",index=False)

In [ ]:
import ee
import pandas as pd

# Initialize the Earth Engine API
ee.Initialize(project="ee-bhupendrarulekiller14")

# Function to get weather data for a given latitude, longitude, and date
def get_weather_data(lat, lon, date):
    # Load the weather dataset (e.g., ERA5)
    weather_collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
    
    # Filter by the specific date
    weather_collection = weather_collection.filterDate(date, ee.Date(date).advance(1, 'day'))
    
    # Get the latest weather image (or the only image for the given date)
    latest_image = weather_collection.sort('system:time_start', False).first()
    
    # Define the point geometry
    point = ee.Geometry.Point([lon, lat])
    
    # Extract weather data directly from the image
    weather_data = latest_image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=1000
    ).getInfo()
    
    # Return temperature, wind speed, and precipitation
    return {
        "temperature": weather_data.get('temperature_2m', None),  # Adjust band name based on dataset
        "windspeed": weather_data.get('u_component_of_wind_10m', None),  # Adjust band name
        "precipitation": weather_data.get('total_precipitation', None)  # Adjust band name
    }

# Iterate over the DataFrame and fetch weather data for each row
weather_results = []
for _, row in df.iterrows():
    # Extract date from 'File Creat' column and convert it to the required format (YYYY-MM-DD)
    file_date = pd.to_datetime(row['File Creat']).strftime('%Y-%m-%d')
    
    # Fetch weather data for the row's lat, lon, and the file date
    try:
        weather_info = get_weather_data(row['lat'], row['lon'], file_date)
    except Exception as e:
        print(f"Error fetching data for lat={row['lat']}, lon={row['lon']}: {e}")
        weather_info = {"temperature": None, "windspeed": None, "precipitation": None}
    
    weather_results.append(weather_info)

# Add the weather data back to the DataFrame
weather_df = pd.DataFrame(weather_results)
df = pd.concat([df.reset_index(drop=True), weather_df], axis=1)

# Save the updated DataFrame
df.to_csv("weather_data.csv", index=False)

print("Weather data added to DataFrame!")
